In [3]:
!pip3 install git+https://github.com/tira-io/tira.git@development#\&subdirectory=python-client

  Cloning https://github.com/tira-io/tira.git (to revision development) to /tmp/pip-req-build-5f_zl5ng
  Running command git clone --filter=blob:none --quiet https://github.com/tira-io/tira.git /tmp/pip-req-build-5f_zl5ng
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
  Resolved https://github.com/tira-io/tira.git to commit 7dd0e7216943ad2844b62c36939b60920e034a2d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for tira: filename=tira-0.0.134-py3-none-any.whl size=94772 sha256=29574058872a17e5514c6b0b99492ada1ffd160ac647583fd1a8b0189a8005e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-w_6csnyj/wheels/1f/36/7a/0f641175add91ee2acac0e59ab6ed9b3b040

In [1]:
from tira.rest_api_client import Client
import pandas as pd

tira = Client()

In [2]:
APPROACHES = [
    # tinyfsu
    'reneuir-2024/tinyfsu/tiny-fsu-bert',
    'reneuir-2024/tinyfsu/cross-encoder-ms-marco-TinyBERT-L-2',
    'reneuir-2024/tinyfsu/tiny-bert-ranker-bm25',
    'reneuir-2024/tinyfsu/tiny-bert-ranker-l2-bm25',
    'reneuir-2024/tinyfsu/tiny-bert-ranker-distillation',
    'reneuir-2024/tinyfsu/tiny-bert-ranker-l-2-distillation',
    
    #tira ir starter
    'reneuir-2024/naverlabseurope/Splade (re-ranker)',
    'reneuir-2024/tira-ir-starter/MonoT5 Base (tira-ir-starter-gygaggle)',
    'reneuir-2024/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)',
    'reneuir-2024/tira-ir-starter/TASB msmarco-distilbert-base-cos (tira-ir-starter-beir)',
    'reneuir-2024/tira-ir-starter/ANCE Base Dot (tira-ir-starter-beir)',
    'reneuir-2024/tira-ir-starter/MonoT5 3b (tira-ir-starter-gygaggle)',
    'reneuir-2024/tira-ir-starter/MonoBERT Large (tira-ir-starter-gygaggle)',
    #'reneuir-2024/tira-ir-starter/MonoBERT Small (tira-ir-starter-gygaggle)',
    #SBERT multi-qa-mpnet-base-dot-v1 (tira-ir-starter-beir)
    #SBERT msmarco-distilbert-dot-v5 (tira-ir-starter-beir)
    #ANCE as bi encoder
    
    
    # fschlatt
    'reneuir-2024/fschlatt/sparse-cross-encoder-4-512',
    'reneuir-2024/fschlatt/castorini-list-in-t5-150',
    'reneuir-2024/fschlatt/rank-zephyr',
    # TODO: monoelectra
    
    
    # Sean
    # TODO: Add approach presented from workshop
    
    # Eugene
    # TODO: Add approach presented from workshop
    
    # Shengyao
    # TODO: Add approach presented from workshop
    
    # Xueguang
    # TODO: Add approach presented from workshop
    
    # 
    # 6x query expansion with Llama Flant5 >> BM25
    
    # ToDo: 5 PyTerrier approaches
    
    # ToDo: 5 Anserini approaches
]

In [7]:
df = []

for approach in APPROACHES:
    profiling = tira.profiling.from_submission(approach, 'dl-top-1000-docs-20240701-training')
    elapsed_time = profiling[-1]
    assert elapsed_time['key'] == 'elapsed_time'
    elapsed_time = elapsed_time['value']
    rss = [i['value'] for i in profiling if i['key'] == 'ps_vsz']
    gpu_memory_used = [int(i['value'].split()[0]) for i in profiling if i['key'] == 'gpu_memory_used']
    gpu_utilization = [int(i['value'].split()[0]) for i in profiling if i['key'] == 'gpu_utilization']
    cpu_utilization = [i['value'] for i in profiling if i['key'] == 'ps_cpu']
    df += [{
        'approach': approach.split('2024/')[1],
        'Elapsed Time': elapsed_time,
        'CPU Utilization (Max)': max(cpu_utilization),
        'RSS (MAX)': max(rss),
        'GPU Memory (Max)': max(gpu_memory_used),
        'GPU Utilization (Max)': max(gpu_utilization),
    }]

df = pd.DataFrame(df)
df.sort_values('Elapsed Time')

,approach,Elapsed Time,CPU Utilization (Max),RSS (MAX),GPU Memory (Max),GPU Utilization (Max)
0,tinyfsu/tiny-fsu-bert,64.0,135.7,17957372.0,250,23
1,tinyfsu/cross-encoder-ms-marco-TinyBERT-L-2,65.0,134.7,17953288.0,250,24
3,tinyfsu/tiny-bert-ranker-l2-bm25,77.0,210.7,17965852.0,250,23
2,tinyfsu/tiny-bert-ranker-bm25,100.0,135.7,17962184.0,250,14
9,tira-ir-starter/TASB msmarco-distilbert-base-c...,243.0,121.1,23648928.0,6380,100
4,tinyfsu/tiny-bert-ranker-distillation,343.0,104.4,28967756.0,2600,81
13,fschlatt/sparse-cross-encoder-4-512,360.0,143.2,394142648.0,3238,100
5,tinyfsu/tiny-bert-ranker-l-2-distillation,395.0,104.1,28966740.0,2600,72
10,tira-ir-starter/ANCE Base Dot (tira-ir-starter...,462.0,120.1,23146064.0,5880,100
8,tira-ir-starter/ColBERT Re-Rank (tira-ir-start...,910.0,113.4,27281316.0,2350,100
